# ¿Que más se podría hacer?

In [1]:
%load_ext autoreload
%autoreload 2

- Test con data augmentation y la LSTM
- Usar los tokenizadores usados en los transformers para ver como funcionan
- TSNE de embeddings y analizar si hay interpretación
- Probar con otras encimas o proteinas
- Usar los embeddings entrenados para analizar resultados de proteinas o encimas con menos data
- Entrenar una red neuronal con los features (fingerprints por ejemplo) y comparar los resultados con los embeddings


# Tome cualquiera de estas propuestas o alguna suya y desarrolle

<b>Entreno una red neuronal utilizando los featres provistos en el dataset</b>

In [19]:
import pandas as pd
import numpy as np


## Obtención de datos para entrenamiento

In [32]:
DATA_PATH = 'data/acetylcholinesterase_02_bioactivity_data_preprocessed_full.pkl'

In [33]:
try:

    df = pd.read_pickle(DATA_PATH)

except FileNotFoundError:

    print("Preparando dataset...\n")
    
    from chembl_webresource_client.new_client import new_client

    target = new_client.target
    target_query = target.search('acetylcholinesterase')
    targets = pd.DataFrame.from_dict(target_query)


    selected_target = targets.target_chembl_id[0]
    print(f"selected_target: {selected_target}\n")


    activity = new_client.activity
    res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

    res_cols = []
    for i, r in enumerate(res):
        print(f'downloaded: {i}\r', end='')
        res_cols.append(r)

    df = pd.DataFrame(res_cols)

    df = df.dropna(subset=['standard_value', 'canonical_smiles'])
    df = df.drop_duplicates(['canonical_smiles'])

    df['standard_value'] = df['standard_value'].apply(pd.to_numeric)
    df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x>1e8)*1e8 + (x<=1e8)*x)
    df['pIC50'] = df['standard_value'].apply(lambda x: -np.log10(x*(10**-9)))

    # Elimino datos no numericos de la columna target. De otro modo luego ocasionan inconvenientes durante 
    # el proceso de entrenamiento
    df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

    df.to_pickle(DATA_PATH)
    print("\nDataset descargado y listo para trabajar")

## Completar

In [35]:
df.head(3)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,standard_value_norm,pIC50
0,None,33969,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,9606,None,None,IC50,uM,UO_0000065,None,0.75,750.0,6.124939
1,None,37563,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,9606,None,None,IC50,uM,UO_0000065,None,0.1,100.0,7.000000
2,None,37565,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,9606,None,None,IC50,uM,UO_0000065,None,50.0,50000.0,4.301030


In [37]:
df.head(1).to_dict("records")

[{'activity_comment': None,
  'activity_id': 33969,
  'activity_properties': [],
  'assay_chembl_id': 'CHEMBL643384',
  'assay_description': 'Inhibitory concentration against acetylcholinesterase',
  'assay_type': 'B',
  'assay_variant_accession': None,
  'assay_variant_mutation': None,
  'bao_endpoint': 'BAO_0000190',
  'bao_format': 'BAO_0000357',
  'bao_label': 'single protein format',
  'canonical_smiles': 'CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1',
  'data_validity_comment': None,
  'data_validity_description': None,
  'document_chembl_id': 'CHEMBL1148382',
  'document_journal': 'J. Med. Chem.',
  'document_year': 2004.0,
  'ligand_efficiency': {'bei': '19.61',
   'le': '0.36',
   'lle': '3.32',
   'sei': '9.21'},
  'molecule_chembl_id': 'CHEMBL133897',
  'molecule_pref_name': None,
  'parent_molecule_chembl_id': 'CHEMBL133897',
  'pchembl_value': '6.12',
  'potential_duplicate': 0,
  'qudt_units': 'http://www.openphacts.org/units/Nanomolar',
  'record_id': 252547,
  'relation': '=',